<a href="https://colab.research.google.com/github/caomy7/JRMP_Tutorial/blob/main/JRMP_2025_Tutorial4_AT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pip install

In [ ]:
!pip install geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.8 MB/s eta 0:00:00


# initialize

In [ ]:
import ee
import geemap
ee.Authenticate()
# ee.Initialize()
ee.Initialize(project='adroit-dock-424302-f2')

# AT

In [10]:

import ee
import geemap
import pandas as pdc
import ee
import geemap
import datetime


# 初始化Earth Engine
# ee.Authenticate()
# ee.Initialize()
Map = geemap.Map()


Map.add_basemap('HYBRID')

roi = ee.FeatureCollection("projects/ee-my-gee2025/assets/Boston")


vegIndex = 'EVI2'
th = 0.2
threshMin = 0.1
scale = 50

year1 = "2020" ###c
lag = 20
startDate = year1+'-01-01'
endDate = year1+'-12-31'


point = ee.Geometry.Point([-71.067898, 42.35526])

dataset = ee.ImageCollection("COPERNICUS/S2").filterBounds(roi).filter(ee.Filter.date(startDate, endDate))



def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;

  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).select("B.*").copyProperties(image,["system:time_start"])


filtered = dataset.map(maskS2clouds)

# rgbVis = {
#   "bands": ['B11', 'B8', 'B3'],
#   "min": 0,
#   "max": 3000
# };

# Map.addLayer(filtered,rgbVis,"sentinel_cloudfree")
# Map
# Map.addLayerControl()
# Map

def EVI2(image):
  evi2 = image.expression(
    '2.5*(NIR-RED) / (NIR+2.4*RED+1)',{
      'RED': image.select('B4').multiply(0.0001),
      'NIR': image.select('B8').multiply(0.0001),
    })

  return image.addBands(evi2.rename('EVI2'))

withEVI2 = filtered.map(EVI2)

# collection = withEVI2.select('EVI2')

# Map
vis = {"min": 0, "max": 1, "palette": [
  'FFFFFF', 'CE7E45', 'FCD163', '66A000', '207401',
  '056201', '004C00', '023B01', '012E01', '011301'
]}

# Map.addLayer(withEVI2.select('EVI2'),vis,"EVI2")
# Map.centerObject(roi,7)
# Map.addLayerControl()
# Map

# SECTION 4 - Generate composites

precipitation = withEVI2.select('EVI2')


ft = ee.FeatureCollection(ee.List([]))


listDates = ee.List.sequence(ee.Date(startDate).millis(), ee.Date(endDate).millis(), 86400000*lag)

#  map the dates
def dds(dd):
  date_window = ee.Date(ee.Number(dd))
  date_startW = date_window.advance(-lag/2, 'days')
  date_endW = date_window.advance(lag/2, 'days')
  col_window = precipitation.filterDate(date_startW,date_endW)
  out = col_window.reduce(ee.Reducer.mean())

  return out.set('system:time_start',date_window.millis()).set('empty', col_window.size().eq(0))


# var colDekads = colDekads.map(function(im){

# colDekads = colDekads.map(lambda im:im)
# colDekadsRaw = ee.ImageCollection(listDates.map(dds).flatten()
colDekadsRaw = ee.ImageCollection(listDates.map(dds).flatten())


# colDekadsRaw
def fill_evi2_min(im):
    return ee.Image(threshMin).rename('EVI2_mean').copyProperties(im, ['system:time_start'])

colDekadsEmpty = colDekadsRaw.filterMetadata('empty', 'equals', 1).map(fill_evi2_min)

colDekads = colDekadsRaw.filterMetadata('empty', 'equals', 0).merge(colDekadsEmpty)


def moving_average_gap_filling(im):
    lag = 20
    date_window = ee.Date(im.get('system:time_start'))
    date_startW = date_window.advance(-lag*2, 'days')
    date_endW = date_window.advance(lag*2, 'days')


    meanIm1 = colDekads.filterDate(date_startW,date_window.advance(1, 'days')).reduce(ee.Reducer.mean())
    meanIm2 = colDekads.filterDate(date_window.advance(-1, 'days'),date_endW).reduce(ee.Reducer.mean())

    meanIm = (meanIm1.add(meanIm2)).divide(2)

    return im.unmask(meanIm).copyProperties(im, ['system:time_start'])
    # return im.unmask(meanIm).copyProperties(im, ['system:time_start'])

colDekads = colDekads.map(moving_average_gap_filling)
colDekads = colDekads.sort('system:time_start')
# Map.addLayer(colDekads.first(),vis,"colDekads")

# # Map.addLayer(colDekads,vis,"colDekads")
# Map
# Map.addLayerControl()
# Map

# SECTION 2 - Define function for cubic interpolation


def cubicInterpolation(collection,step):
    listDekads = ee.List.sequence(1,collection.size().subtract(3),1)


    def col_inter(ii):
        ii = ee.Number(ii)
        p0 = ee.Image(collection.toList(10000).get(ee.Number(ii).subtract(1)))
        p1 = ee.Image(collection.toList(10000).get(ii))
        p2 = ee.Image(collection.toList(10000).get(ee.Number(ii).add(1)))
        p3 = ee.Image(collection.toList(10000).get(ee.Number(ii).add(2)))


        diff01 = ee.Date(p1.get('system:time_start')).difference(ee.Date(p0.get('system:time_start')), 'day')
        diff12 = ee.Date(p2.get('system:time_start')).difference(ee.Date(p1.get('system:time_start')), 'day')
        diff23 = ee.Date(p3.get('system:time_start')).difference(ee.Date(p2.get('system:time_start')), 'day')


        diff01nor = diff01.divide(diff12)
        diff12nor = diff12.divide(diff12)
        diff23nor = diff23.divide(diff12)


        f0 = p1
        f1 = p2
        f0p = (p2.subtract(p0)).divide(diff01nor.add(diff12nor))
        f1p = (p3.subtract(p1)).divide(diff12nor.add(diff23nor))


        a = (f0.multiply(2)).subtract(f1.multiply(2)).add(f0p).add(f1p)
        b = (f0.multiply(-3)).add(f1.multiply(3)).subtract(f0p.multiply(2)).subtract(f1p)
        c = f0p
        d = f0

        xValues = ee.List.sequence(0,diff12.subtract(1),step)
        xDates = ee.List.sequence(p1.get('system:time_start'),p2.get('system:time_start'),86400000)


        def divide_fun(x):
            im = ee.Image(ee.Number(x).divide(diff12))
            return (im.pow(3)).multiply(a).add((im.pow(2)).multiply(b)).add(im.multiply(c)).add(d).set('system:time_start',ee.Number(xDates.get(x)))

        interp = xValues.map(divide_fun)
        return interp


    # colInterp = listDekads.map(col_inter)
    colInterp = listDekads.map(col_inter)
    print(type(colInterp))
    colInterp = ee.ImageCollection(colInterp.flatten())

    return colInterp


# SECTION 6 - Interpolate time series

interp = cubicInterpolation(colDekads,1)

# SECTION 7 - Estimate phenology metrics


init = ee.Image(ee.Date(str((int(year1)-1))+'-12-31').millis());

def EVI2_interp(im):
  return im.rename('EVI2_interp').addBands(im.metadata('system:time_start','date1')).set('system:time_start', im.get('system:time_start'))

interp = interp.map(EVI2_interp)  #add time band

minND = ee.Image(threshMin)
maxND = colDekads.max()
amplitude = maxND.subtract(minND)

thresh = amplitude.multiply(th).add(minND).rename('EVI2_interp')

def th_interp(im):
  out = im.select('EVI2_interp').gt(thresh);
  return im.updateMask(out).copyProperties(im,['system:time_start'])


col_aboveThresh = interp.map(th_interp)

def clip(image):
  clip = image.clip(roi)
  return image.updateMask(clip)

col_aboveThresh = col_aboveThresh.map(clip)


SoS = col_aboveThresh.reduce(ee.Reducer.firstNonNull()).select('date1_first').rename('SoS')
SoS_doy = SoS.subtract(init).divide(86400000);

EoS = col_aboveThresh.reduce(ee.Reducer.lastNonNull()).select('date1_last').rename('EoS')
EoS_doy = EoS.subtract(init).divide(86400000);

# print(type(SoS_doy))
# print(type(EoS_doy))


phenoPalette1 = ['ffffff',"76ee00","7fff00","76ee00","66cd00","458b00","22bb22",'4aff00','008000',"006400"]
phenoPalette2 = ['ffffff',"ffffe0","fffacd","fafad2",'ffff00',"ffff00","ffd700","ffa500",'ffbc00','ff4500']

visSoS = {min:120,max:200,"palette":phenoPalette1}
visEoS = {min:180,max:300,"palette":phenoPalette2}

Map.addLayer(SoS_doy,visSoS,'SoS')
Map.addLayer(EoS_doy,visEoS,'EoS')
Map.addLayer(roi,{},'ROI')
# Map.add_colorbar_branca(colors=phenoPalette1, vmin=120, vmax=200, layer_name="Start of Season")
# Map.add_colorbar_branca(colors=phenoPalette2, vmin=180, vmax=300, layer_name="End of Season")
Map.addLayerControl()
Map.centerObject(point,11)

Map


<class 'ee.ee_list.List'>


Map(center=[42.35526, -71.067898], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…